In [1]:
import websocket
import sys
# from websocket import create_connection

import simplejson as json

sys.path.append('../trainedModel_historical\MetaTrader5')


<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
C:\Users\tshol\AppData\Local\Temp\ipykernel_8456\310442952.py:7: SyntaxWarning: invalid escape sequence '\M'
  sys.path.append('../trainedModel_historical\MetaTrader5')


In [2]:

# ws = websocket.create_connection("wss://api.tiingo.com/fx")

# subscribe = {
#         'eventName':'subscribe',
#         'authorization':'ebed6535a2acbe1a06b0057e8d1e8af61e5cefa7',
#         'eventData': {
#             'thresholdLevel': 5,
#             'tickers': ['eurusd']
#     }
# }

# ws.send(json.dumps(subscribe))
# while True:
#     print(ws.recv())

In [ ]:
import websocket
import json
import pandas as pd
from datetime import datetime

# Create an empty DataFrame to store real-time data
df = pd.DataFrame(columns=['ticker', 'datetime', 'bidPrice', 'askPrice', 'midPrice'])

# WebSocket connection
ws = websocket.create_connection("wss://api.tiingo.com/fx")

# Subscription message
subscribe = {
    'eventName': 'subscribe',
    'authorization': 'ebed6535a2acbe1a06b0057e8d1e8af61e5cefa7',  # Replace with your actual API key
    'eventData': {
        'thresholdLevel': 5,
        'tickers': ['eurusd']
    }
}

# Send subscription message
ws.send(json.dumps(subscribe))

# Function to process received data and append it to DataFrame
def process_message(message):
    global df
    data = json.loads(message)
    
    # Only process price update messages (messageType == 'A')
    if data.get('service') == 'fx' and data.get('messageType') == 'A':
        fx_data = data['data']
        
        # Extract relevant fields from the message
        ticker = fx_data[1]  # Ticker (e.g., 'eurusd')
        datetime_str = fx_data[2]  # Date as string
        bid_price = fx_data[4]  # Bid price
        ask_price = fx_data[6]  # Ask price
        mid_price = fx_data[5]  # Mid price (calculated as (bid + ask) / 2)
        
        # Convert datetime string to a proper datetime object (optional)
        timestamp = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%f%z")
        
        # Append the data to the DataFrame
        new_row = {
            'ticker': ticker,
            'datetime': timestamp,
            'bidPrice': bid_price,
            'askPrice': ask_price,
            'midPrice': mid_price
        }
        df = df.append(new_row, ignore_index=True)

        # Optionally, display the updated DataFrame or save to a file
        print(df.tail(1))  # Display the last few rows

# Main loop to receive data and process it
while True:
    message = ws.recv()
    # print(message)
    process_message(message)

